# 章節 15：維度處理

## 15.7 還有一個問題

多數的迭代對總似然性沒有任何有用的貢獻。當觀察到的物種種類 m 變大的時，抽樣到不可忽視的似然性的機率變得很小。真的非常小。但幸運的，有解決方法。

如果你觀察到一組資料，你可以一次用所有的資料更新先驗分佈，或是依次序更新資料，最後的結果是相同的。

本節的例子來說，關鍵是每一次更新一種種類。這樣當我們隨機取樣一組樣本 ps, 只有一種種類影響似然性的運算，所以選到一個好的樣本機會大很多。

以下是一次更新一種種類的版本：

<pre>
# 繼承自 Species , 所以假設組是一序列的 Dirichlet 物件（而不像 Species2）
class Species4(Species):
    def Update(self, data):
        m = len(data)
        
        # 迭代所有觀察到的種類
        for i in range(m):
        
            # 產生此種類的資料
            one = numpy.zeros(i+1)
            one[i] = data[i]
            
            # 呼叫父類別的 Update 更新每個 Dirichlet Object
            Species.Update(self, one)
</pre>

所以上例中，我們做三次更新。
1. 我看到了三隻獅子
2. 我看到兩隻老虎且沒有任何獅子
3. 我看到一隻熊且沒有任何獅子跟老虎

下方是 Species4 的 Likelihood 方法：
<pre>
# class Species4
    def Likelihood(self, data, hypo):
        dirichlet = hypo
        like = 0
        for i in range(self.iterations):
            like += dirichlet.Likelihood(data)
            # correct for the number of unseen species the new one
            # could have been
            m = len(data)
            num_unseen = dirichlet.n - m + 1
            like *= num_unseen
        return like
</pre>

這幾乎跟 Species.Likelihood 方法相同。差別在於 num_unseen 因子。這個修正是必要的，因為每一次我們看到一個種類，我們還需要考慮有多少沒見到但有可能見到的種類。對於較大的 n 值，我們可以看到更多看不見的物種，會增加資料的似然性。

這是一個細微的觀點，作者承認一開始沒有用對。但作者可以透過與之前版本的比較來驗證此版本的正確性。